In [647]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, make_scorer, f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [648]:
df = pd.read_csv('../Dataset/SelectedFeatures-10s-TOR-NonTOR.csv')

In [649]:
print(df.describe(), '\n')

        Source Port   Destination Port      Protocol   Flow Duration  \
count  67834.000000       67834.000000  67834.000000    6.783400e+04   
mean   37912.753324       11566.395967     12.167291    2.991884e+06   
std    20986.077326       18374.765123      5.459410    4.063005e+06   
min       21.000000          21.000000      6.000000    0.000000e+00   
25%    19305.000000         137.000000      6.000000    4.435975e+04   
50%    43677.000000         443.000000     17.000000    4.108570e+05   
75%    54685.000000       16311.000000     17.000000    7.325550e+06   
max    65534.000000       65514.000000     17.000000    1.000000e+07   

        Flow Bytes/s   Flow Packets/s   Flow IAT Mean   Flow IAT Std  \
count   6.783200e+04     6.783400e+04    6.783400e+04   6.783400e+04   
mean             inf              inf    3.155927e+05   2.209662e+05   
std              NaN              NaN    6.988069e+05   6.409506e+05   
min     0.000000e+00     2.002581e-01    0.000000e+00   0.00000

In [650]:
print(df.head())

   Source IP   Source Port  Destination IP   Destination Port   Protocol  \
0  10.0.2.15         53913   216.58.208.46                 80          6   
1  10.0.2.15         53913   216.58.208.46                 80          6   
2  10.0.2.15         53913   216.58.208.46                 80          6   
3  10.0.2.15         53913   216.58.208.46                 80          6   
4  10.0.2.15         53913   216.58.208.46                 80          6   

    Flow Duration   Flow Bytes/s   Flow Packets/s   Flow IAT Mean  \
0             435            0.0      4597.701149           435.0   
1             259            0.0      7722.007722           259.0   
2             891            0.0      2244.668911           891.0   
3            1074            0.0      1862.197393          1074.0   
4             315            0.0      6349.206349           315.0   

    Flow IAT Std  ...   Bwd IAT Min  Active Mean   Active Std   Active Max  \
0            0.0  ...             0            0  

In [651]:
df.shape

(67834, 29)

In [652]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67834 entries, 0 to 67833
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Source IP          67834 non-null  object 
 1    Source Port       67834 non-null  int64  
 2    Destination IP    67834 non-null  object 
 3    Destination Port  67834 non-null  int64  
 4    Protocol          67834 non-null  int64  
 5    Flow Duration     67834 non-null  int64  
 6    Flow Bytes/s      67832 non-null  float64
 7    Flow Packets/s    67834 non-null  float64
 8    Flow IAT Mean     67834 non-null  float64
 9    Flow IAT Std      67834 non-null  float64
 10   Flow IAT Max      67834 non-null  int64  
 11   Flow IAT Min      67834 non-null  int64  
 12  Fwd IAT Mean       67834 non-null  float64
 13   Fwd IAT Std       67834 non-null  float64
 14   Fwd IAT Max       67834 non-null  int64  
 15   Fwd IAT Min       67834 non-null  int64  
 16  Bwd IAT Mean       678

In [653]:
print(df['label'].value_counts())

label
nonTOR    59790
TOR        8044
Name: count, dtype: int64


In [654]:
print(df.isnull().sum())

Source IP            0
 Source Port         0
 Destination IP      0
 Destination Port    0
 Protocol            0
 Flow Duration       0
 Flow Bytes/s        2
 Flow Packets/s      0
 Flow IAT Mean       0
 Flow IAT Std        0
 Flow IAT Max        0
 Flow IAT Min        0
Fwd IAT Mean         0
 Fwd IAT Std         0
 Fwd IAT Max         0
 Fwd IAT Min         0
Bwd IAT Mean         0
 Bwd IAT Std         0
 Bwd IAT Max         0
 Bwd IAT Min         0
Active Mean          0
 Active Std          0
 Active Max          0
 Active Min          0
Idle Mean            0
 Idle Std            0
 Idle Max            0
 Idle Min            0
label                0
dtype: int64


In [655]:
categorical_features = ["Source IP", " Destination IP"]
numeric_features = df.columns.difference(categorical_features + ["label"])

In [656]:
df.fillna(0, inplace=True)

In [657]:
upper_limit = 1e9
lower_limit = -1e9
df[numeric_features] = df[numeric_features].applymap(lambda x: min(upper_limit, max(lower_limit, x)))

In [658]:
print(df['label'].value_counts())

label
nonTOR    59790
TOR        8044
Name: count, dtype: int64


In [659]:
duplicati = df[df.duplicated()]
numero_duplicati = df.duplicated().sum()
print('Numero di duplicati: ', numero_duplicati)

Numero di duplicati:  396


In [660]:
df.drop_duplicates()

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,...,Bwd IAT Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,label
0,10.0.2.15,53913,216.58.208.46,80,6,435,0.000000e+00,4.597701e+03,435.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
1,10.0.2.15,53913,216.58.208.46,80,6,259,0.000000e+00,7.722008e+03,259.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
2,10.0.2.15,53913,216.58.208.46,80,6,891,0.000000e+00,2.244669e+03,891.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
3,10.0.2.15,53913,216.58.208.46,80,6,1074,0.000000e+00,1.862197e+03,1074.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
4,10.0.2.15,53913,216.58.208.46,80,6,315,0.000000e+00,6.349206e+03,315.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67828,10.8.8.134,34251,134.170.18.137,443,6,0,1.000000e+09,1.000000e+09,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
67829,131.202.240.183,7116,239.255.255.250,1900,17,0,1.000000e+09,1.000000e+09,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
67831,131.202.240.87,11365,31.13.73.1,443,6,0,1.000000e+09,1.000000e+09,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR
67832,131.202.6.26,13000,131.202.240.87,64584,6,0,0.000000e+00,1.000000e+09,0.0,0.0,...,0,0,0,0,0,0,0,0,0,nonTOR


In [661]:
unique_value_columns = df[numeric_features].nunique() == 1
columns_to_drop = unique_value_columns[unique_value_columns].index
numeric_features = [col for col in numeric_features if col not in columns_to_drop]
df = df.drop(columns=columns_to_drop)
df.shape

(67834, 27)

In [662]:
correlation_matrix = df[numeric_features].corr()
print(correlation_matrix)

                    Active Max   Active Min   Bwd IAT Max   Bwd IAT Min  \
 Active Max           1.000000     1.000000      0.258841      0.021367   
 Active Min           1.000000     1.000000      0.258841      0.021367   
 Bwd IAT Max          0.258841     0.258841      1.000000      0.336722   
 Bwd IAT Min          0.021367     0.021367      0.336722      1.000000   
 Bwd IAT Std          0.229780     0.229780      0.813613      0.011486   
 Destination Port     0.014742     0.014742      0.061794      0.077405   
 Flow Bytes/s        -0.003909    -0.003909     -0.008080     -0.003471   
 Flow Duration        0.192200     0.192200      0.418185      0.103096   
 Flow IAT Max         0.370279     0.370279      0.718126      0.243098   
 Flow IAT Mean        0.253244     0.253244      0.072690      0.202059   
 Flow IAT Min         0.198153     0.198153     -0.109417     -0.027436   
 Flow IAT Std         0.231191     0.231191      0.476827      0.398644   
 Flow Packets/s      -0.0

In [663]:
def correlation(df, threshold):
    col_corr = set()  
    corr_matrix = df.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold:
                colname = corr_matrix.columns[i]
                col_corr.add(colname)
    return col_corr

highly_correlated_features = correlation(df[numeric_features], 0.8)
print("Feature altamente correlate:", highly_correlated_features)

Feature altamente correlate: {' Bwd IAT Std', ' Flow IAT Std', ' Active Min', ' Flow IAT Min', ' Idle Min', ' Fwd IAT Max', 'Bwd IAT Mean', 'Fwd IAT Mean', ' Fwd IAT Std', 'Idle Mean', 'Active Mean'}


In [664]:
df.drop(columns=highly_correlated_features, inplace=True)

In [665]:
top_n_categories = 25
for column in categorical_features:
    top_categories = df[column].value_counts().nlargest(top_n_categories).index
    df[column] = df[column].where(df[column].isin(top_categories), 'Other')

In [666]:
encoder = OneHotEncoder(sparse=False, drop='first') 
df_encoded = pd.DataFrame(encoder.fit_transform(df[categorical_features]), columns=encoder.get_feature_names_out(categorical_features))

df = pd.concat([df, df_encoded], axis=1)

df.drop(categorical_features, axis=1, inplace=True)

c:\Users\Tommaso\Desktop\uni\Hackaton\.conda\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [667]:
print(df.shape)

(67834, 64)


In [668]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('label', axis=1), df['label'], test_size=0.2, random_state=42
)

In [669]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [670]:
df_balanced = pd.DataFrame(X_train_resampled, columns=X_train.columns)
df_balanced['label'] = y_train_resampled

class_counts = df_balanced['label'].value_counts()

print("Numero di campioni per classe:")
print(class_counts)

Numero di campioni per classe:
label
nonTOR    47794
TOR       47794
Name: count, dtype: int64


In [671]:
print(X_train_resampled.shape)

(95588, 63)


In [672]:
n_samples = len(X_train_resampled)  

max_samples = int(1 * n_samples)

In [673]:
model = IsolationForest(contamination=0.5, n_estimators=1000, random_state=42, max_samples=max_samples, max_features=0.8,  n_jobs=-1)

In [674]:
model.fit(X_train_resampled)

IsolationForest(contamination=0.5, max_features=0.8, max_samples=95588,
                n_estimators=1000, n_jobs=-1, random_state=42)

In [675]:
predictions = model.predict(X_test)

In [676]:
label_mapping = {'TOR': 1, 'nonTOR': -1}
y_test_numeric = y_test.map(label_mapping)

In [677]:
print("Confusion Matrix:")
print(confusion_matrix(y_test_numeric, predictions))

print("\nClassification Report:")
print(classification_report(y_test_numeric, predictions))

Confusion Matrix:
[[10364  1632]
 [  152  1419]]

Classification Report:
              precision    recall  f1-score   support

          -1       0.99      0.86      0.92     11996
           1       0.47      0.90      0.61      1571

    accuracy                           0.87     13567
   macro avg       0.73      0.88      0.77     13567
weighted avg       0.93      0.87      0.89     13567



feature selection